In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
import pandas as pd
import os, json

def save_json(file, data):
    file_path = f"{file}.json"
    if os.path.exists(file_path):
        with open(file_path, "r") as f:
            existing_data = json.load(f)
        existing_data.extend(data)
        with open(file_path, "w") as f:
            f.write(json.dumps(existing_data, indent=2))
    else:
        try:
            with open(file_path, "w") as f:
                f.write(json.dumps(data, indent=2))
        except Exception as e:
            print(e)

In [3]:
import requests
from bs4 import BeautifulSoup
import json


headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36'}

# Replace with your target URL
url = "https://biblicalcounseling.com/resource-library/topic-index/"
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
categories = {}
category_sections = soup.find_all('li', class_='collection__item')
for section in category_sections:
    category_name = section.find('a',class_="collection__item--title").text.strip()        
    links = section.find_all('li', class_='term_child')
    # Store links for this category
    categories[category_name] = [{
        'title': link.text.strip(),
        'url': link.find("a")["href"]
    } for link in links]

In [4]:
def scrape_content(link, category, subcategory):
    # url = "https://biblicalcounseling.com"
    # print(link)

    response = requests.get(f'{link}', headers=headers)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')
    header_component = soup.find("div", class_= "card-content")
    title = header_component.find("h1").text.strip() if header_component.find("h1") else " " 
    subtitle = header_component.find("h3").text.strip() if header_component.find("h3") else " " 
    question = header_component.find("p").text.strip() if header_component.find("p") else " " 
    authors = header_component.find("div", class_="authors-wrapper").text.strip() if header_component.find("div", class_="authors-wrapper") else " "
    content = soup.find("div", class_="article-content").text.strip()
    source = url

    return {
            "title": title,
            "subtitle": subtitle,
            "author": authors,
            "category": category,
            "subcategory": subcategory,
            "question": question,
            "body": content,
            "source": source
        }

In [5]:
categories

{'ACBC': [{'title': 'Announcements and Events 33',
   'url': '/resource-library/topic-index/announcements-and-events'},
  {'title': 'Book Reviews 59',
   'url': '/resource-library/topic-index/book-reviews'},
  {'title': 'Certification 6',
   'url': '/resource-library/topic-index/certification'}],
 'Biblical Counseling and Psychology': [{'title': 'Medical Issues 9',
   'url': '/resource-library/topic-index/medical-issues'},
  {'title': 'Mental Health 19',
   'url': '/resource-library/topic-index/mental-health'},
  {'title': 'Psychological Theories 24',
   'url': '/resource-library/topic-index/psychological-theories'}],
 'Counseling Practice': [{'title': 'Counseling Methodology 53',
   'url': '/resource-library/topic-index/counseling-methodology'},
  {'title': 'How to Begin a Counseling Center 16',
   'url': '/resource-library/topic-index/how-to-begin-a-counseling-center'},
  {'title': 'Legal Issues in Counseling 12',
   'url': '/resource-library/topic-index/legal-issues-in-counseling'}]

In [6]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36'}
base_url = "https://biblicalcounseling.com"

data_list_dict = []

for k, v in  list(categories.items())[8:]:
    for vals in v:
        response = requests.get(f'{base_url}{vals['url']}', headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        articles = soup.find("div", class_="card-list")
        articles_links = articles.find_all("a", class_="overlink")
        articles_links = [f"{link['href']}" for link in articles_links]
        for article in articles_links:
            # print(article)
            data = scrape_content(link=article, category=k, subcategory=vals["title"])
            print(data)
            data_list_dict.append(data)          
        
        save_json("biblical_counseling", data_list_dict)
        # print(articles_links)

{'title': 'Should I Take the Pill?', 'subtitle': 'Truth In Love 403', 'author': 'Dale Johnson  Sean Perron', 'category': 'Typical Issues', 'subcategory': 'Abortion 8', 'question': 'Should birth control pills be considered an abortifacient?', 'body': 'Dale Johnson: This week on the podcast, I am delighted to have my friend, Sean Perron. He has an M.Div from the Southern Baptist Theological Seminary, he serves as an associate pastor at First Baptist Church in Jacksonville, Florida, he’s an ACBC-certified counselor with a specialization in marriage counseling. He’s also the author of three books: Letters to a Romantic, a whole series, one on dating, one on marriage, and one on the first years of marriage. He’s currently pursuing a PhD in Applied Theology and Sean and I are getting to work together on that wonderful project, which I’m thoroughly excited about and I pray that you will hear more about later as well. And then he’s focusing on a theology emphasis with biblical counseling here 